# Clustering

*Justin R. Garrard*

### *Executive Summary*

This section represents the **Data Preparation** and **Modeling** sections of the CRISP-DM process.


### *Objectives*


1. **[Feature Selection]** To prototype the selection of features and data processing required before clustering.


2. **[Clustering Experimentation]** To prototype the clustering process, experimenting until a satisfactory result is produced.



### Setup

In [1]:
# Import libraries
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from ipywidgets import * 
from sklearn.cluster import KMeans

In [2]:
# Declare global variables
DATA_DIR = os.path.join('../data/processed')
DATA_FILE = os.path.join(DATA_DIR, 'processed.csv')
plt.style.use('ggplot')

CLUSTERING_COLS = ['leaid', 'year', 'lea_name', 'fips', 'number_of_schools', 'cbsa_type', 
                  'teachers_total_fte', 'spec_ed_students', 
                  'enrollment_x',
                  'read_test_num_valid', 'read_test_pct_prof_midpt', 'math_test_num_valid',
                   'math_test_pct_prof_midpt', 'rev_total', 'exp_total']

## Set a target year for early analysis
TGT_YEAR = 2016

In [6]:
# Useful functions
def null_counter(df):
    record_nulls = []
    for col in df.columns:
        nulls = df[col].isnull().sum()
        percent_null = round((nulls / df.shape[0]) * 100, 2)
        record_nulls.append([col, nulls, percent_null])
    output = pd.DataFrame(record_nulls, columns=['Attribute', 'Null Count', '% Null'])
    return output

def get_year_range(df):
    year_range = list(df['year'].unique())
    year_range.sort()
    return year_range

def subset_by_states_only(df):
    df = df[df['fips'] <= 56]
    return df

def sound_off(df):
    nRow, nCol = cluster_df.shape
    print(f'There are {nRow} rows and {nCol} columns.')
    print('')

    YEAR_RANGE = get_year_range(cluster_df)
    print(f'Data spans the years {YEAR_RANGE[0]} to {YEAR_RANGE[-1]}.')
    print('')

    print('Available columns include:')
    display(null_counter(cluster_df))
    
def filter_out_factor(df, column_name):
    ## Identify records with null values in column
    bad_records = df[df[column_name].isnull()]
    bad_records.to_csv(f'missing_{column_name}.csv')

    ## Drop records with null values in column
    df = df[df[column_name].notnull()]
    return df

### Data Preparation

In this section we load the data, running various scripts to format the contents properly.

***High-Level Overview***

We tried to choose a subset of columns in which the data was mostly complete. That meant disqualifying rows that were:

* ... not states (i.e. territories).


* ... did not have reported scores for standardized tests.


We were especially disappointed to have to remove "english_language_learners" from the clustering data. In literature this factor is frequently referred to as significant. But, more than 6,000 records in our limited set simply have no reported value for this metric. 

In [4]:
# Load and preview data
## Isolate by specific columns
cluster_df = pd.read_csv(DATA_FILE)[CLUSTERING_COLS]
## Filter out non-state records
cluster_df = subset_by_states_only(cluster_df)
## Filter by year
cluster_df = cluster_df[cluster_df['year'] == TGT_YEAR]

sound_off(cluster_df)

/home/roy/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,9,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


There are 18654 rows and 15 columns.

Data spans the years 2016 to 2016.

Available columns include:


,Attribute,Null Count,% Null
0,leaid,0,0.00
1,year,0,0.00
2,lea_name,0,0.00
3,fips,0,0.00
4,number_of_schools,0,0.00
5,cbsa_type,1,0.01
6,teachers_total_fte,885,4.74
7,spec_ed_students,2238,12.00
8,enrollment_x,1383,7.41
9,read_test_num_valid,2374,12.73


In [7]:
# Remove records with missing test scores
cluster_df = filter_out_factor(cluster_df, 'read_test_num_valid')
cluster_df = filter_out_factor(cluster_df, 'math_test_num_valid')
sound_off(cluster_df)

There are 16272 rows and 15 columns.

Data spans the years 2016 to 2016.

Available columns include:


,Attribute,Null Count,% Null
0,leaid,0,0.00
1,year,0,0.00
2,lea_name,0,0.00
3,fips,0,0.00
4,number_of_schools,0,0.00
5,cbsa_type,0,0.00
6,teachers_total_fte,31,0.19
7,spec_ed_students,467,2.87
8,enrollment_x,37,0.23
9,read_test_num_valid,0,0.00


In [8]:
# Remove records with missing spec_ed_students data
cluster_df = filter_out_factor(cluster_df, 'spec_ed_students')
sound_off(cluster_df)

There are 15805 rows and 15 columns.

Data spans the years 2016 to 2016.

Available columns include:


,Attribute,Null Count,% Null
0,leaid,0,0.00
1,year,0,0.00
2,lea_name,0,0.00
3,fips,0,0.00
4,number_of_schools,0,0.00
5,cbsa_type,0,0.00
6,teachers_total_fte,28,0.18
7,spec_ed_students,0,0.00
8,enrollment_x,6,0.04
9,read_test_num_valid,0,0.00


In [9]:
# Remove records with missing teachers_total_fte and enrollment_x
cluster_df = filter_out_factor(cluster_df, 'teachers_total_fte')
cluster_df = filter_out_factor(cluster_df, 'enrollment_x')
sound_off(cluster_df)

There are 15772 rows and 15 columns.

Data spans the years 2016 to 2016.

Available columns include:


,Attribute,Null Count,% Null
0,leaid,0,0.0
1,year,0,0.0
2,lea_name,0,0.0
3,fips,0,0.0
4,number_of_schools,0,0.0
5,cbsa_type,0,0.0
6,teachers_total_fte,0,0.0
7,spec_ed_students,0,0.0
8,enrollment_x,0,0.0
9,read_test_num_valid,0,0.0


In [10]:
# Remove the columns that won't be used as features
cluster_prepared_df = cluster_df.drop(['leaid', 'year', 'lea_name', 'fips'], axis=1)

### Clustering

The purpose of this tool is specifically *descriptive* analytics. In short, we are looking to understand our underlying data, rather than build predictions.


In [23]:
# Setup a KMeans learner
## Parameters
clusters = 8
random_seed = 777

## Constructor
kmeans_learner = KMeans(n_clusters=clusters, random_state=random_seed)

In [24]:
# Fit the KMeans learner with the available data
results = kmeans_learner.fit_predict(cluster_prepared_df)

In [25]:
# Display the results, looking for patterns
print(results.shape)
print()
print(results[0])

(15772,)

0


In [26]:
# Attach the labels to the original dataframe
cluster_df['labels'] = results

In [28]:
# View the characteristics of each labeled dataset
for i in range(0, clusters):
    subset = cluster_df[cluster_df['labels'] == i]
    print(i)
    display(subset.describe())
    print()

0


,leaid,year,fips,number_of_schools,cbsa_type,teachers_total_fte,spec_ed_students,enrollment_x,read_test_num_valid,read_test_pct_prof_midpt,math_test_num_valid,math_test_pct_prof_midpt,rev_total,exp_total,labels
count,1.308500e+04,13085.0,13085.000000,13085.000000,13085.000000,13085.000000,13085.000000,13085.000000,13085.000000,13085.000000,13085.000000,13085.000000,1.308500e+04,1.308500e+04,13085.0
mean,3.010011e+06,2016.0,29.983645,3.014597,0.411769,71.577153,159.549026,1088.980741,567.932060,50.009744,566.779060,44.463279,1.371731e+07,1.351898e+07,0.0
std,1.437162e+06,0.0,14.367338,3.292272,1.489271,125.473898,424.919548,1966.599519,969.655554,19.606918,980.509037,21.710719,1.311025e+07,1.297747e+07,0.0
min,1.000050e+05,2016.0,1.000000,0.000000,-2.000000,0.000000,3.000000,0.000000,1.000000,-3.000000,1.000000,-3.000000,-2.000000e+00,-2.000000e+00,0.0
25%,1.918030e+06,2016.0,19.000000,1.000000,-2.000000,19.590000,36.000000,278.000000,135.000000,36.000000,135.000000,28.000000,3.757000e+06,3.624000e+06,0.0
50%,3.013560e+06,2016.0,30.000000,2.000000,1.000000,45.000000,87.000000,647.000000,337.000000,51.000000,337.000000,42.000000,8.908000e+06,8.700000e+06,0.0
75%,4.023550e+06,2016.0,40.000000,4.000000,1.000000,94.840000,199.000000,1416.000000,742.000000,65.000000,739.000000,60.000000,1.989600e+07,1.954300e+07,0.0
max,5.606240e+06,2016.0,56.000000,117.000000,2.000000,4044.660000,17373.000000,62815.000000,30703.000000,99.500000,31107.000000,99.500000,6.480200e+07,6.521900e+07,0.0



1


,leaid,year,fips,number_of_schools,cbsa_type,teachers_total_fte,spec_ed_students,enrollment_x,read_test_num_valid,read_test_pct_prof_midpt,math_test_num_valid,math_test_pct_prof_midpt,rev_total,exp_total,labels
count,1.200000e+01,12.0,12.000000,12.000000,12.0,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,1.200000e+01,1.200000e+01,12.0
mean,2.470779e+06,2016.0,24.666667,292.750000,1.0,13374.658333,25767.416667,214392.666667,118422.000000,51.916667,120495.916667,49.916667,2.760856e+09,2.848292e+09,1.0
std,1.509334e+06,0.0,15.041357,91.051559,0.0,3827.102007,8120.370832,70948.782071,43897.358262,13.701084,43705.233958,16.962302,5.036405e+08,5.810640e+08,0.0
min,1.200180e+06,2016.0,12.000000,206.000000,1.0,7951.870000,14887.000000,130814.000000,64640.000000,28.000000,65782.000000,20.000000,2.121511e+09,2.057207e+09,1.0
25%,1.201298e+06,2016.0,12.000000,220.750000,1.0,11336.682500,19222.750000,175915.000000,92319.500000,48.500000,92858.750000,41.250000,2.314538e+09,2.337748e+09,1.0
50%,1.950255e+06,2016.0,19.500000,282.000000,1.0,12483.810000,24666.000000,196697.500000,114972.000000,52.000000,117560.500000,52.500000,2.777372e+09,2.840278e+09,1.0
75%,3.454792e+06,2016.0,34.500000,316.250000,1.0,16299.575000,31556.500000,230042.500000,133671.000000,54.250000,132664.750000,58.000000,3.121526e+09,3.136256e+09,1.0
max,5.101260e+06,2016.0,51.000000,528.000000,1.0,20884.000000,38604.000000,357249.000000,212368.000000,84.000000,211532.000000,77.000000,3.679802e+09,3.785166e+09,1.0



2


...

,leaid,year,fips,number_of_schools,cbsa_type,teachers_total_fte,spec_ed_students,enrollment_x,read_test_num_valid,read_test_pct_prof_midpt,math_test_num_valid,math_test_pct_prof_midpt,rev_total,exp_total,labels
count,3.800000e+01,38.0,38.000000,38.000000,38.0,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,3.800000e+01,3.800000e+01,38.0
mean,2.902620e+06,2016.0,28.947368,133.710526,1.0,5733.942895,11177.394737,91664.263158,48394.684211,48.657895,48899.868421,47.078947,1.248235e+09,1.279461e+09,5.0
std,1.679499e+06,0.0,16.787942,44.827542,0.0,2109.286438,4027.488598,33947.642568,19318.664196,18.711575,20130.750422,20.421745,2.833956e+08,2.915687e+08,0.0
min,6.145500e+05,2016.0,6.000000,59.000000,1.0,2715.000000,5334.000000,34293.000000,14319.000000,17.000000,14392.000000,15.000000,8.579880e+08,9.334540e+08,5.0
25%,1.300412e+06,2016.0,13.000000,106.250000,1.0,4190.240000,8440.750000,73874.000000,36247.250000,33.250000,36247.000000,28.250000,1.032429e+09,1.027427e+09,5.0
50%,2.451605e+06,2016.0,24.500000,126.500000,1.0,5648.675000,10379.000000,88386.500000,47237.500000,48.000000,50540.000000,46.500000,1.199210e+09,1.230440e+09,5.0
75%,4.782500e+06,2016.0,47.750000,167.250000,1.0,6864.125000,13412.500000,110088.500000,58812.000000,56.500000,58609.000000,58.500000,1.396518e+09,1.423654e+09,5.0
max,5.509600e+06,2016.0,55.000000,240.000000,1.0,11125.200000,21822.000000,178214.000000,95602.000000,87.000000,109581.000000,90.000000,1.972752e+09,1.937798e+09,5.0



6


,leaid,year,fips,number_of_schools,cbsa_type,teachers_total_fte,spec_ed_students,enrollment_x,read_test_num_valid,read_test_pct_prof_midpt,math_test_num_valid,math_test_pct_prof_midpt,rev_total,exp_total,labels
count,2.006000e+03,2006.0,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2.006000e+03,2.006000e+03,2006.0
mean,2.918350e+06,2016.0,29.051346,10.999501,1.044865,394.077856,875.807577,6379.858923,3324.984546,55.764955,3327.952642,50.954885,9.344832e+07,9.398240e+07,6.0
std,1.511047e+06,0.0,15.124714,6.106086,0.531296,185.865776,434.634266,3167.335882,1812.951963,17.366784,1820.292745,19.097925,3.284032e+07,3.327737e+07,0.0
min,1.000060e+05,2016.0,1.000000,1.000000,-2.000000,0.000000,3.000000,254.000000,32.000000,3.000000,26.000000,2.000000,3.854700e+07,4.844400e+07,6.0
25%,1.721855e+06,2016.0,17.000000,7.000000,1.000000,262.875000,572.000000,4061.500000,2045.000000,43.000000,2044.250000,37.250000,6.590525e+07,6.574975e+07,6.0
50%,3.401485e+06,2016.0,34.000000,10.000000,1.000000,357.180000,791.000000,5645.500000,2952.500000,56.000000,2967.000000,51.000000,8.454350e+07,8.550200e+07,6.0
75%,4.200046e+06,2016.0,42.000000,14.000000,1.000000,483.337500,1099.000000,8186.750000,4347.250000,69.000000,4361.750000,66.000000,1.152725e+08,1.159602e+08,6.0
max,5.605830e+06,2016.0,56.000000,78.000000,2.000000,1405.860000,3078.000000,19475.000000,12382.000000,97.000000,12424.000000,98.000000,1.851640e+08,2.083000e+08,6.0



7


,leaid,year,fips,number_of_schools,cbsa_type,teachers_total_fte,spec_ed_students,enrollment_x,read_test_num_valid,read_test_pct_prof_midpt,math_test_num_valid,math_test_pct_prof_midpt,rev_total,exp_total,labels
count,1.0,1.0,1.0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.000000e+00,1.000000e+00,1.0
mean,1709930.0,2016.0,17.0,585.0,1.0,19016.08,51826.0,378199.0,184139.0,28.0,185465.0,24.0,5.840203e+09,5.827667e+09,7.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1709930.0,2016.0,17.0,585.0,1.0,19016.08,51826.0,378199.0,184139.0,28.0,185465.0,24.0,5.840203e+09,5.827667e+09,7.0
25%,1709930.0,2016.0,17.0,585.0,1.0,19016.08,51826.0,378199.0,184139.0,28.0,185465.0,24.0,5.840203e+09,5.827667e+09,7.0
50%,1709930.0,2016.0,17.0,585.0,1.0,19016.08,51826.0,378199.0,184139.0,28.0,185465.0,24.0,5.840203e+09,5.827667e+09,7.0
75%,1709930.0,2016.0,17.0,585.0,1.0,19016.08,51826.0,378199.0,184139.0,28.0,185465.0,24.0,5.840203e+09,5.827667e+09,7.0
max,1709930.0,2016.0,17.0,585.0,1.0,19016.08,51826.0,378199.0,184139.0,28.0,185465.0,24.0,5.840203e+09,5.827667e+09,7.0


In [29]:
# Output labeled data
cluster_df.to_csv('processed_labeled.csv')